In [42]:
import httpx
import shutil
import sqlite3
import uuid

In [6]:
USERNAME = shutil.os.getenv("USER")
FIREFOX_COOKIE_PATH = f"/Users/{USERNAME}/Library/Application Support/Firefox/Profiles/brqcjaua.default-release/cookies.sqlite"
def get_cookie(domain: str):
    shutil.copy(FIREFOX_COOKIE_PATH, "./cookies.sqlite")
    connector = sqlite3.connect("cookies.sqlite")
    cursor = connector.cursor()
    cursor.execute(f"SELECT name, value FROM moz_cookies WHERE host = '{domain}';")
    cookies = {name: value for name, value in cursor.fetchall()}
    connector.close()
    return cookies

In [7]:
cookies = get_cookie(".lusha.com")

In [11]:
r = httpx.get('https://dashboard-services.lusha.com/api/v1/account-assets/balances/display', cookies=cookies)

In [ ]:
print(f"""Credits:
Available = {r.json()['shared'][0]['total']}
Used = {r.json()['shared'][0]['totalUsed']}
Remaining = {r.json()['shared'][0]['total'] - r.json()['shared'][0]['totalUsed']}
""")

In [78]:
contacts = [{
        'id': contact.get('id', ''),
        'job_title': contact.get('job_title', {}).get('raw', ''),
        'social_link': contact.get('social_link', ''),
        'first_name': contact.get('name', {}).get('first', ''),
        'last_name': contact.get('name', {}).get('first', '')} for contact in r.json()['contacts']['results']]

In [128]:
def get_contacts(search):
    r = httpx.post('https://dashboard-services.lusha.com/v2/prospecting-full', cookies=cookies, json={
        'filters': {'searchText': [search]}, # mandatory
        'filtersMetadata': {'isViewEmployeesMode': False},  # optional
        'display': 'contacts', # optional
        'pages': {'page': 0, 'pageSize': 25}, # optional
        'sessionId': str(uuid.uuid4()), # mandatory
        'searchTrigger': 'NewTab', # mandatory
        'savedSearchId': 0, # mandatory
        'bulkSearchCompanies': {}, # optional
        'isRecent': False, # optional
        'isSaved': False, # optional
        'pageAbove400': None, # optional
        'totalPagesAbove400': 0, # optional
        'excludeRevealedContacts': False # optional
    })
    contacts = []
    for result in r.json()['contacts']['results']:
        contact = {
            'id': result.get('id', ''),
            'contact_id': result.get('contactId', ''),
            'contact_input_id': result.get('contactInputId', ''),
            'job_title': result.get('job_title', {}).get('raw', ''),
            'social_link': result.get('social_link', ''),
            'first_name': result.get('name', {}).get('first', ''),
            'last_name': result.get('name', {}).get('last', ''),
            'emails': [], 'phones': [], 'hidden_emails': [], 'hidden_phones': []}
        for email in result.get('emails', []):
            if email.get('status', '') == 'shown':
                contact['emails'].append(email.get('address', ''))
            else:
                contact['hidden_emails'].append(email.get('id', ''))
        for phone in result.get('phones', []):
            if phone.get('status', '') == 'shown':
                contact['phones'].append(phone.get('number', ''))
            else:
                contact['hidden_phones'].append(phone.get('id', ''))
        contacts.append(contact)
    return contacts, {'mask_id': r.json()['maskId'], 'request_id': r.json()['requestId']}

In [129]:
contacts, session = get_contacts('Arthur')

In [132]:
def unmask_emails(mask_id: str, request_id: str, contact):
    r = httpx.post('https://dashboard-services.lusha.com/v1/api/shown-contacts/unmask', cookies=cookies, json={
        'maskId': mask_id,
        'contacts': [{
            'uniqueId': contact['id'],
            'contact_id': contact['contact_id'],
            'contact_input_id': contact['contact_input_id'],
            'phone_data_points_ids': [],
            'email_data_points_ids': contact['hidden_emails']}],
        'requestId': request_id,
        'product': 'prospecting-full',
        'useBulkCredits': False})
    emails = []
    for contact in r.json()['data']['data']['contacts']:
        for email in contact['emails']:
            emails.append(email['address'])
    return emails

In [121]:
def print_contact(contact):
    print(f"""{contact['first_name']} {contact['last_name']} (hidden emails: {len(contact['hidden_emails'])}, hidden phones: {len(contact['hidden_phones'])})""")

In [ ]:
print_contact(contacts[12])

In [ ]:
unmask_emails(session['mask_id'], session['request_id'], contacts[12])